In [1]:
import py7zr
import os
import shutil
import subprocess

In [2]:
# find the 7z.exe
def find7z():
    path = shutil.which("7z.exe")
    if path is None:
        # check if 7z.exe is in the default installation directory
        if os.path.exists("C:\\Program Files\\7-Zip\\7z.exe"):
            path = "C:\\Program Files\\7-Zip\\7z.exe"
            return path
        elif os.path.exists("C:\\Program Files (x86)\\7-Zip\\7z.exe"):
            path = "C:\\Program Files (x86)\\7-Zip\\7z.exe"
            return path
        else:
            print("7z.exe not found, please specify the path to 7z.exe")
            return None
    return path

In [3]:
path = find7z()
if path is None:
    print("7z.exe not found")

In [4]:
# get password list 
def getPasswordList():
    passwordList = []
    with open("./passwords.txt", "r") as f:
        for line in f:
            passwordList.append(line.strip())
        passwordList.append("")
    return passwordList

In [14]:
passwords = getPasswordList()

# unzip a file
def unzipFileWith7z(file, passwordList=passwords, autodelete=False, autodeleteexisting=False, lv=0, maximum_lv=2):
    has_archive = False
    password_protected = False

    # check if the file exists
    if not os.path.exists(file):
        print(f"File {file} does not exist")
        return

    # check if is a .lib file, if yes skip
    # TODO: maybe a temporary solution, need to find a way to unzip .lib files
    if file.endswith(".lib"):
        print(f"File {file} is a .lib file, skipping...")
        return

    # check if the output directory already exists
    if os.path.exists(f"{file}lv{lv:d}"):
        if autodeleteexisting:
            print(f"Output directory {file}lv{lv:d} already exists, deleting...")
            shutil.rmtree(f"{file}lv{lv:d}")
        else:
            print(f"Output directory {file}lv{lv:d} already exists, skipping...")
            return

    # unzip without password
    result = subprocess.run([path, "x", "-p", file,  f"-o{file}lv{lv:d}"], capture_output=True,  stdin=subprocess.DEVNULL)
    if result.returncode == 2 and result.stderr.decode("utf-8", errors="replace").find("Wrong password") != -1:
        print(f"Archive {file} is password protected")
        shutil.rmtree(f"{file}lv{lv:d}", ignore_errors=True)
        has_archive = True
        password_protected = True
    if result.returncode == 2 and result.stderr.decode("utf-8", errors="replace").find("Cannot open the file as archive") != -1:
        if lv == 0:
            print(f"File \"{file}\" is not an archive")
        shutil.rmtree(f"{file}lv{lv:d}", ignore_errors=True)
        return False
    if result.returncode == 0:
        print(f"Archive {file} is not password protected, unzipped to {file}lv{lv:d}")
        has_archive = True

    # unzip with password
    if password_protected:
        for password in passwords:
            result = subprocess.run([path, "x", f"-p{password}", file,  f"-o{file}lv{lv:d}"], capture_output=True,  stdin=subprocess.DEVNULL)
            if result.returncode == 2 and result.stderr.decode("utf-8", errors="replace").find("Wrong password") != -1:
                shutil.rmtree(f"{file}lv{lv:d}", ignore_errors=True)
            else:
                print(f"Correct password for {file} is {password}, unzipped to {file}lv{lv:d}")
                has_archive = True
                break
    
    if has_archive:
        files = os.listdir(f"{file}lv{lv:d}")

        # check if the file just unzipped is a normal file
        is_normal_dll = 0
        if '.rdata' in files: 
            is_normal_dll += 1
        if '.text' in files:
            is_normal_dll += 1
        if '.data' in files:
            is_normal_dll += 1
        if '.idata' in files:
            is_normal_dll += 1
        if is_normal_dll >= 2:
            print(f"File {file} is a normal file, skipping...")
            shutil.rmtree(path=f"{file}lv{lv:d}", ignore_errors=True)
            return False, lv
        
        # check if last level is reached
        # criteria 1 : files just extracted contains at the same time .dll and .exe files (it is a program)
        contains_dll = False
        contains_exe = False
        for root, dirs, files in os.walk(f"{file}lv{lv:d}"):
            for file_ in files:
                if file_.endswith(".dll"):
                    contains_dll = True
                if file_.endswith(".exe"):
                    contains_exe = True
        if contains_dll and contains_exe:
            print(f"File {file} is a program, will not go to next level")
            return False, lv

        # criteria 2 : files just extracted contains at the same time .exe files and folders (it is a program without .dll files)
        contains_folder = False
        if any(os.path.isdir(os.path.join(f"{file}lv{lv:d}", item)) for item in os.listdir(f"{file}lv{lv:d}")):
            contains_folder = True
        
        if contains_exe and contains_folder:
            print(f"File {file} is a program, will not go to next level")
            return False, lv

        # other files without .dll or .exe files will automatically stop at the next level

        print(f"Archive {file} has been unzipped to {file}lv{lv:d}, going to next level")
        lv += 1
        # recursively unzip the files in the directory just created
        for root, dirs, files in os.walk(f"{file}lv{lv-1:d}"):
            for file in files:
                unzipFileWith7z(os.path.join(root, file), passwordList, autodelete, autodeleteexisting, lv)
        return True, lv
    return False, lv
    

In [15]:
for root, dirs, files in os.walk("./test/"):
    for file in files:
        unzipFileWith7z(os.path.join(root, file), autodeleteexisting=True, )

Output directory ./test/NO.071\0071第2层压缩.01lv0 already exists, deleting...
The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.
Archive ./test/NO.071\0071第2层压缩.01 is password protected
Correct password for ./test/NO.071\0071第2层压缩.01 is www.acgbuluo.com, unzipped to ./test/NO.071\0071第2层压缩.01lv0
Archive ./test/NO.071\0071第2层压缩.01 has been unzipped to ./test/NO.071\0071第2层压缩.01lv0, going to next level
Archive ./test/NO.071\0071第2层压缩.01lv0\NO.07110月会员partme_第1层压缩.rar is password protected
Correct password for ./test/NO.071\0071第2层压缩.01lv0\NO.07110月会员partme_第1层压缩.rar is www.acgbuluo.com, unzipped to ./test/NO.071\0071第2层压缩.01lv0\NO.07110月会员partme_第1层压缩.rarlv1
Archive ./test/NO.071\0071第2层压缩.01lv0\NO.07110月会员partme_第1层压缩.rar has been unzipped to ./test/NO.071\0071第2层压缩.01lv0\NO.07110月会员partme_第1层压缩.rarlv1, going to next level
Archive ./test/NO.071\0071第2层压缩.01lv0\NO.07110月会员partme_第1层压缩.rar is password 

FileNotFoundError: [WinError 3] The system cannot find the path specified: './test/NO.071\\0071第2层压缩.01lv0\\NO.07110月会员partme_第1层压缩.rarlv0'

In [ ]:
# unzip archives under a certain directory
def unzipDirectory(directory, passwordList=[""], autoDelete=False):
    # list files and directories under the directory
    entries = os.listdir(directory)

    # loop through the entries
    for count, entry in enumerate(entries):
        # get full path of the entry
        fullPath = os.path.join(directory, entry)

        # check if the entry is a file or directory
        if os.path.isdir(fullPath):
            # if directory, recursively call this function
            unzipDirectory(fullPath, passwordList, autoDelete)
        else:
            # if file, unzip it
            unzipFile(fullPath, passwordList)

            # if autoDelete is True, delete the file
            if autoDelete:
                os.remove(fullPath)

    return


In [ ]:
# unzip a single volume archive 
def unzipFile(file, passwordList=[""]):

    return

# unzip a multi-volume archive 
def unzipMultiVolumeFile(files, passwordList=[""]):
    return


def checkIfMultiVolume(file):
    return